# PRISM Pattern Design Tutorial

Learn how to design and compare different sampling patterns for optimal image reconstruction.

This tutorial covers:
1. Understanding sampling patterns
2. Implementing custom pattern functions
3. Comparing pattern performance
4. Optimizing coverage

**Estimated time**: 25-30 minutes

## 1. Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# PRISM imports
from prism.core.patterns import generate_fermat_spiral


%matplotlib inline
plt.rcParams["figure.figsize"] = (15, 10)
plt.rcParams["figure.dpi"] = 100

print("Setup complete!")

## 2. Pattern Generator Functions

Let's define several common sampling pattern generators to compare:

In [ ]:
# Define pattern generator functions


def fermat_spiral(n: int, radius: float) -> np.ndarray:
    """Generate Fermat spiral pattern using PRISM's built-in function."""
    points = generate_fermat_spiral(n_points=n, r_max=radius, length=0)
    return points.squeeze().numpy()


def grid_pattern(n: int, radius: float) -> np.ndarray:
    """Generate a grid pattern within a circular region."""
    # Determine grid size (approximate sqrt)
    grid_size = int(np.ceil(np.sqrt(n)))

    # Create grid points
    x = np.linspace(-radius, radius, grid_size)
    y = np.linspace(-radius, radius, grid_size)
    xx, yy = np.meshgrid(x, y)
    points = np.column_stack([xx.ravel(), yy.ravel()])

    # Filter points within radius
    distances = np.linalg.norm(points, axis=1)
    points = points[distances <= radius]

    # Take only n points if we have more
    if len(points) > n:
        indices = np.linspace(0, len(points) - 1, n, dtype=int)
        points = points[indices]

    return points


def circular_pattern(n: int, radius: float, n_rings: int = 5) -> np.ndarray:
    """Generate concentric circular pattern."""
    points = []
    points_per_ring = n // n_rings

    for i in range(n_rings):
        r = radius * (i + 1) / n_rings
        n_points_ring = max(4, points_per_ring)
        angles = np.linspace(0, 2 * np.pi, n_points_ring, endpoint=False)
        for angle in angles:
            points.append([r * np.cos(angle), r * np.sin(angle)])

    return np.array(points[:n])


def random_pattern(n: int, radius: float, seed: int = 42) -> np.ndarray:
    """Generate random pattern within circular region."""
    np.random.seed(seed)
    points = []

    while len(points) < n:
        x = np.random.uniform(-radius, radius)
        y = np.random.uniform(-radius, radius)
        if np.sqrt(x**2 + y**2) <= radius:
            points.append([x, y])

    return np.array(points)


# Configuration
n_samples = 50
radius = 100

# Generate patterns
patterns = {
    "Fermat Spiral": fermat_spiral(n=n_samples, radius=radius),
    "Grid": grid_pattern(n=n_samples, radius=radius),
    "Circular": circular_pattern(n=n_samples, radius=radius),
    "Random": random_pattern(n=n_samples, radius=radius),
}

# Visualize all patterns
fig, axes = plt.subplots(2, 2, figsize=(15, 15))
axes = axes.flatten()

for idx, (name, points) in enumerate(patterns.items()):
    ax = axes[idx]
    scatter = ax.scatter(
        points[:, 0], points[:, 1], c=range(len(points)), cmap="viridis", s=100, alpha=0.7
    )
    ax.set_title(f"{name} ({len(points)} points)", fontsize=14, fontweight="bold")
    ax.set_xlabel("X position")
    ax.set_ylabel("Y position")
    ax.axis("equal")
    ax.grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=ax, label="Sample order")

plt.tight_layout()
plt.show()

print(f"Generated {len(patterns)} different sampling patterns")

## 3. Custom Pattern: Logarithmic Spiral

Let's implement a custom pattern - a logarithmic spiral:

In [ ]:
def logarithmic_spiral(n: int, radius: float, growth_rate: float = 0.1) -> np.ndarray:
    """
    Generate points along a logarithmic spiral.

    Parameters
    ----------
    n : int
        Number of points
    radius : float
        Maximum radius
    growth_rate : float
        How quickly the spiral expands (default: 0.1)

    Returns
    -------
    np.ndarray
        Points array of shape (n, 2)
    """
    points = []

    # Determine number of turns needed
    max_theta = np.log(radius) / growth_rate
    thetas = np.linspace(0, max_theta, n)

    for theta in thetas:
        r = growth_rate * np.exp(growth_rate * theta)
        x = r * np.cos(theta)
        y = r * np.sin(theta)
        points.append([x, y])

    return np.array(points)


# Test the custom pattern
log_spiral = logarithmic_spiral(n=50, radius=100, growth_rate=0.15)

plt.figure(figsize=(10, 10))
plt.scatter(log_spiral[:, 0], log_spiral[:, 1], c=range(len(log_spiral)), cmap="plasma", s=100)
plt.colorbar(label="Sample order")
plt.title("Custom Logarithmic Spiral Pattern", fontsize=16, fontweight="bold")
plt.xlabel("X position")
plt.ylabel("Y position")
plt.axis("equal")
plt.grid(True, alpha=0.3)
plt.show()

print(f"Created custom pattern with {len(log_spiral)} points")

## 4. Pattern Metrics

Let's define metrics to quantitatively compare patterns:

In [ ]:
def compute_pattern_metrics(points: np.ndarray) -> dict:
    """
    Compute metrics for a sampling pattern.

    Returns
    -------
    dict
        Dictionary containing:
        - min_distance: Minimum distance between adjacent points
        - mean_distance: Mean distance between adjacent points
        - coverage_radius: Effective coverage radius
        - uniformity: Measure of pattern uniformity (lower is better)
    """
    from scipy.spatial.distance import pdist, squareform

    # Pairwise distances
    distances = pdist(points)
    dist_matrix = squareform(distances)

    # Replace diagonal zeros with inf
    np.fill_diagonal(dist_matrix, np.inf)

    # Nearest neighbor distances
    nearest_neighbor_dists = np.min(dist_matrix, axis=1)

    # Compute metrics
    metrics = {
        "n_points": len(points),
        "min_distance": float(np.min(nearest_neighbor_dists)),
        "mean_distance": float(np.mean(nearest_neighbor_dists)),
        "std_distance": float(np.std(nearest_neighbor_dists)),
        "coverage_radius": float(np.max(np.linalg.norm(points, axis=1))),
        "uniformity": float(np.std(nearest_neighbor_dists) / np.mean(nearest_neighbor_dists)),
    }

    return metrics


# Compute metrics for all patterns
print("Pattern Metrics Comparison:\n" + "=" * 70)
print(f"{'Pattern':<20} {'Points':>8} {'Min Dist':>12} {'Mean Dist':>12} {'Uniformity':>12}")
print("=" * 70)

all_metrics = {}
for name, points in patterns.items():
    metrics = compute_pattern_metrics(points)
    all_metrics[name] = metrics
    print(
        f"{name:<20} {metrics['n_points']:>8} {metrics['min_distance']:>12.2f} "
        f"{metrics['mean_distance']:>12.2f} {metrics['uniformity']:>12.3f}"
    )

# Add custom pattern
log_metrics = compute_pattern_metrics(log_spiral)
all_metrics["Log Spiral"] = log_metrics
print(
    f"{'Log Spiral':<20} {log_metrics['n_points']:>8} {log_metrics['min_distance']:>12.2f} "
    f"{log_metrics['mean_distance']:>12.2f} {log_metrics['uniformity']:>12.3f}"
)
print("=" * 70)
print("\nLower uniformity = more consistent spacing (better)")

## 5. Coverage Visualization

Visualize how well each pattern covers the imaging area:

In [ ]:
def visualize_coverage(points: np.ndarray, aperture_diameter: int = 32, image_size: int = 256):
    """
    Visualize coverage map for a sampling pattern.
    """
    # Create coverage map
    coverage = np.zeros((image_size, image_size))
    center = image_size // 2

    # Add coverage circles for each sample point
    y_grid, x_grid = np.ogrid[:image_size, :image_size]

    for point in points:
        px, py = point + center  # Translate to image coordinates
        # Create circular mask
        mask = (x_grid - px) ** 2 + (y_grid - py) ** 2 <= (aperture_diameter / 2) ** 2
        coverage[mask] += 1

    # Calculate coverage percentage
    coverage_pct = (coverage > 0).sum() / (image_size**2) * 100

    return coverage, coverage_pct


# Visualize coverage for all patterns
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

aperture_size = 32
img_size = 256

patterns_with_log = {**patterns, "Log Spiral": log_spiral}

for idx, (name, points) in enumerate(patterns_with_log.items()):
    coverage, coverage_pct = visualize_coverage(points, aperture_size, img_size)

    ax = axes[idx]
    im = ax.imshow(coverage, cmap="hot", interpolation="nearest")
    ax.set_title(f"{name}\nCoverage: {coverage_pct:.1f}%", fontsize=12, fontweight="bold")
    ax.axis("off")
    plt.colorbar(im, ax=ax, label="Overlap count")

# Remove extra subplot
fig.delaxes(axes[-1])

plt.tight_layout()
plt.show()

print("Coverage comparison complete!")

## 6. Interactive Pattern Designer

Experiment with pattern parameters:

In [ ]:
# Simple parameter exploration
def explore_fermat_spiral(n_points_list=[30, 50, 100, 150]):
    """
    Explore how number of points affects Fermat spiral coverage.
    """
    fig, axes = plt.subplots(2, 2, figsize=(14, 14))
    axes = axes.flatten()

    for idx, n in enumerate(n_points_list):
        points = fermat_spiral(n=n, radius=100)
        metrics = compute_pattern_metrics(points)

        ax = axes[idx]
        scatter = ax.scatter(points[:, 0], points[:, 1], c=range(len(points)), cmap="viridis", s=80)
        ax.set_title(
            f"N={n} points\n"
            f"Mean dist: {metrics['mean_distance']:.1f}, "
            f"Uniformity: {metrics['uniformity']:.3f}",
            fontsize=11,
        )
        ax.set_xlabel("X position")
        ax.set_ylabel("Y position")
        ax.axis("equal")
        ax.grid(True, alpha=0.3)
        plt.colorbar(scatter, ax=ax, label="Order")

    plt.tight_layout()
    plt.show()


# Run the exploration
explore_fermat_spiral()

print("\nObservation: More points = better coverage, but diminishing returns")

## 7. Recommendations

Based on the analysis:

### Pattern Selection Guide

- **Fermat Spiral**: Best overall choice for most cases
  - ✓ Excellent uniformity
  - ✓ Good coverage
  - ✓ Efficient sampling order

- **Grid**: Use when you need structured sampling
  - ✓ Predictable pattern
  - ✗ Poor edge coverage
  - ✗ Aliasing artifacts possible

- **Circular**: Good for radially symmetric objects
  - ✓ Uniform coverage at each radius
  - ✗ Gaps between circles

- **Random**: Avoid for systematic imaging
  - ✗ Poor uniformity
  - ✗ Unpredictable coverage
  - ✓ Good for initial testing

### Optimal Parameters

- **Number of samples**: 50-150 (depending on object complexity)
- **Aperture diameter**: 32-64 pixels (balance between coverage and overlap)
- **Image size**: 512-1024 (quality vs. speed tradeoff)

## 8. Exercises

Try these challenges:

1. **Create a hexagonal pattern** - Implement a function that generates points in a hexagonal grid
2. **Optimize for specific objects** - Design a pattern optimized for Europa's features
3. **Adaptive sampling** - Create a pattern that samples more densely in regions of interest
4. **Coverage optimization** - Find the minimum number of points needed for 90% coverage

## Summary

You've learned:
- ✓ How to visualize and compare sampling patterns
- ✓ How to implement custom pattern functions
- ✓ How to quantify pattern quality with metrics
- ✓ How to optimize coverage

**Next**: Tutorial 3 - Analyzing experiment results